In [23]:
## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))
## Insert whatever audio file you want above

In [27]:
from neo4j import GraphDatabase
import pandas as pd

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        print("Connected to Neo4j.")

    def close(self):
        self.driver.close()
        print("Connection to Neo4j closed.")

    def clear_relationships(self):
        """Removes all CITES and CITED_BY relationships."""
        with self.driver.session(database="DocDatabase") as session:
            try:
                session.run("MATCH ()-[r:CITES]->() DELETE r")
                session.run("MATCH ()-[r:CITED_BY]->() DELETE r")
                print("Removed all CITES and CITED_BY relationships.")
            except Exception as e:
                print(f"Error clearing relationships: {e}")

    def create_outsource_node(self, session, uid, reference_text):
        """Creates a new Outsource node and links it to the source UID."""
        try:
            session.run(
                """
                MATCH (source:Passage {UID: $uid})
                CREATE (o:Outsource {reference_text: $reference_text})
                MERGE (source)-[:OUTSOURCE]->(o)
                """,
                uid=uid,
                reference_text=reference_text
            )
        except Exception as e:
            print(f"Error creating outsource node for UID {uid}: {e}")

    def create_cites_relationship(self, session, source_uid, target_uid, reference_text):
        """Creates CITES and CITED_BY relationships between source and target."""
        try:
            session.run(
                """
                MATCH (source:Passage {UID: $source_uid}), (target:Passage {UID: $target_uid})
                MERGE (source)-[:CITES {reference_text: $reference_text}]->(target)
                MERGE (target)-[:CITED_BY {reference_text: $reference_text}]->(source)
                """,
                source_uid=source_uid,
                target_uid=target_uid,
                reference_text=reference_text
            )
        except Exception as e:
            print(f"Error creating CITES relationship from {source_uid} to {target_uid}: {e}")

    def process_csv(self, csv_path):
        """Processes the CSV file to create relationships in the Neo4j database."""
        try:
            data = pd.read_csv(csv_path)
        except Exception as e:
            print(f"Error reading CSV file {csv_path}: {e}")
            return

        with self.driver.session(database="DocDatabase") as session:
            for index, row in data.iterrows():
                try:
                    source_uid = row['SourceID']
                    reference_text = row['ReferenceText']
                    reference_type = row['ReferenceType']

                    # Skip rows with missing source UID
                    if not source_uid or pd.isna(source_uid):
                        print(f"Skipping row {index}: Missing SourceID.")
                        continue

                    if reference_type == "Outsource":
                        self.create_outsource_node(session, source_uid, reference_text)

                    elif reference_type in ["Internal", "External"]:
                        target_uid = row['TargetID']

                        # Skip rows with missing target UID
                        if not target_uid or pd.isna(target_uid):
                            print(f"Skipping row {index}: Missing TargetID for SourceID {source_uid}.")
                            continue

                        self.create_cites_relationship(session, source_uid, target_uid, reference_text)
                except Exception as e:
                    print(f"Error processing row {index}: {e}")

if __name__ == "__main__":
    # Neo4j connection details
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "12345678"

    # Initialize Neo4j connection
    conn = Neo4jConnection(uri, user, password)

    # Clear old relationships
    conn.clear_relationships()

    # Path to the CSV file
    csv_path = 'CrossReferenceData.csv'

    # Process the CSV file
    conn.process_csv(csv_path)

    # Close the Neo4j connection
    conn.close()


Connected to Neo4j.
Removed all CITES and CITED_BY relationships.
Connection to Neo4j closed.


In [28]:
allDone()

In [17]:
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7687"
user = "neo4j"
password = "12345678"

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        print("Connected to Neo4j.")

    def close(self):
        self.driver.close()
        print("Connection to Neo4j closed.")

    def delete_old_defined_term_relationships(self, batch_size=10000):
        """
        Delete all existing HAS_SHARED_DEFINED_TERM relationships in batches.
        Reduces memory usage for large datasets.
        """
        with self.driver.session(database="DocDatabase") as session:
            while True:
                # Delete a limited number of relationships at a time
                query = f"""
                    MATCH ()-[r:HAS_SHARED_DEFINED_TERM]->()
                    WITH r LIMIT {batch_size}
                    DELETE r
                    RETURN COUNT(r) AS deleted_count
                """
                result = session.run(query)
                deleted_count = result.single()["deleted_count"]
                print(f"Deleted {deleted_count} relationships in this batch.")
                # Break when no more relationships are found
                if deleted_count < batch_size:
                    break
        print("Deleted all old HAS_SHARED_DEFINED_TERM relationships.")

    def get_duplicated_defined_terms(self):
        """Retrieve duplicated named entities and their counts."""
        with self.driver.session(database="DocDatabase") as session:
            query = """
                MATCH (n:DefinedTerm)
                WHERE n.term IS NOT NULL AND n.term <> ""
                WITH n.term AS term, COUNT(*) AS count
                WHERE count > 1
                RETURN term, count
                ORDER BY count DESC
            """
            result = session.run(query)
            duplicated_named_entities = result.data()
            if duplicated_named_entities:
                print("Duplicated Defined Terms and their counts:")
                for record in duplicated_named_entities:
                    print(f"Term: '{record['term']}', Count: {record['count']}")
            else:
                print("No duplicated Defined Term found.")
            return duplicated_named_entities

    def merge_duplicate_defined_terms(self, batch_size=10000):
        """
        Create bidirectional HAS_SHARED_DEFINED_TERM relationships in batches.
        Increases transaction efficiency by processing in smaller chunks.
        """
        with self.driver.session(database="DocDatabase") as session:
            offset = 0
            while True:
                query = f"""
                    MATCH (n1:DefinedTerm), (n2:DefinedTerm)
                    WHERE n1.term = n2.term AND n1.term IS NOT NULL AND n1.term <> "" AND id(n1) < id(n2)
                    WITH n1, n2 SKIP {offset} LIMIT {batch_size}
                    MERGE (n1)-[:HAS_SHARED_DEFINED_TERM]->(n2)
                    MERGE (n2)-[:HAS_SHARED_DEFINED_TERM]->(n1)
                    RETURN COUNT(*) AS processed
                """
                result = session.run(query)
                processed = result.single()["processed"]
                print(f"Processed {processed} relationships in batch {offset // batch_size + 1}")
                if processed < batch_size:
                    break
                offset += batch_size

if __name__ == "__main__":
    # Initialize Neo4j connection
    conn = Neo4jConnection(uri, user, password)

    # Step 1: Delete old HAS_SHARED_DEFINED_TERM relationships
    conn.delete_old_defined_term_relationships()

    # Step 2: Get and print duplicated defined terms
    duplicated_defined_terms = conn.get_duplicated_defined_terms()

    # Step 3: Merge duplicate named entities if any duplicates exist
    if duplicated_defined_terms:
        conn.merge_duplicate_defined_terms()

    # Close the connection
    conn.close()


Connected to Neo4j.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in this batch.
Deleted 10000 relationships in thi

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 111, offset: 152} for query: '\n                    MATCH (n1:DefinedTerm), (n2:DefinedTerm)\n                    WHERE n1.term = n2.term AND n1.term IS NOT NULL AND n1.term <> "" AND id(n1) < id(n2)\n                    WITH n1, n2 SKIP 0 LIMIT 10000\n                    MERGE (n1)-[:HAS_SHARED_DEFINED_TERM]->(n2)\n                    MERGE (n2)-[:HAS_SHARED_DEFINED_TERM]->(n1)\n                    RETURN COUNT(*) AS processed\n                '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {descrip

Processed 10000 relationships in batch 1


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 111, offset: 152} for query: '\n                    MATCH (n1:DefinedTerm), (n2:DefinedTerm)\n                    WHERE n1.term = n2.term AND n1.term IS NOT NULL AND n1.term <> "" AND id(n1) < id(n2)\n                    WITH n1, n2 SKIP 10000 LIMIT 10000\n                    MERGE (n1)-[:HAS_SHARED_DEFINED_TERM]->(n2)\n                    MERGE (n2)-[:HAS_SHARED_DEFINED_TERM]->(n1)\n                    RETURN COUNT(*) AS processed\n                '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {des

Processed 10000 relationships in batch 2


...

Processed 10000 relationships in batch 684


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 111, offset: 152} for query: '\n                    MATCH (n1:DefinedTerm), (n2:DefinedTerm)\n                    WHERE n1.term = n2.term AND n1.term IS NOT NULL AND n1.term <> "" AND id(n1) < id(n2)\n                    WITH n1, n2 SKIP 6840000 LIMIT 10000\n                    MERGE (n1)-[:HAS_SHARED_DEFINED_TERM]->(n2)\n                    MERGE (n2)-[:HAS_SHARED_DEFINED_TERM]->(n1)\n                    RETURN COUNT(*) AS processed\n                '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {d

Processed 10000 relationships in batch 685


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 111, offset: 152} for query: '\n                    MATCH (n1:DefinedTerm), (n2:DefinedTerm)\n                    WHERE n1.term = n2.term AND n1.term IS NOT NULL AND n1.term <> "" AND id(n1) < id(n2)\n                    WITH n1, n2 SKIP 6850000 LIMIT 10000\n                    MERGE (n1)-[:HAS_SHARED_DEFINED_TERM]->(n2)\n                    MERGE (n2)-[:HAS_SHARED_DEFINED_TERM]->(n1)\n                    RETURN COUNT(*) AS processed\n                '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {d

Processed 4835 relationships in batch 686
Connection to Neo4j closed.


In [ ]:
allDone()

In [19]:
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7687"
user = "neo4j"
password = "12345678"

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        print("Connected to Neo4j.")

    def close(self):
        self.driver.close()
        print("Connection to Neo4j closed.")

    def delete_old_named_entity_relationships(self):
        """Delete all existing HAS_SHARED_NAMED_ENTITY relationships."""
        with self.driver.session(database="DocDatabase") as session:
            session.run("MATCH ()-[r:HAS_SHARED_NAMED_ENTITY]->() DELETE r")
            print("Deleted all old HAS_SHARED_NAMED_ENTITY relationships.")

    def get_duplicated_named_entities(self):
        """Retrieve duplicated named entities and their counts."""
        with self.driver.session(database="DocDatabase") as session:
            query = """
                MATCH (n:NamedEntity)
                WHERE n.term IS NOT NULL AND n.term <> ""
                WITH n.term AS term, COUNT(*) AS count
                WHERE count > 1
                RETURN term, count
                ORDER BY count DESC
            """
            result = session.run(query)
            duplicated_named_entities = result.data()
            if duplicated_named_entities:
                print("Duplicated named entities and their counts:")
                for record in duplicated_named_entities:
                    print(f"Term: '{record['term']}', Count: {record['count']}")
            else:
                print("No duplicated named entities found.")
            return duplicated_named_entities

    def merge_duplicate_named_entities(self, batch_size=10000):
        with self.driver.session(database="DocDatabase") as session:
            offset = 0
            while True:
                query = f"""
                    MATCH (n1:NamedEntity), (n2:NamedEntity)
                    WHERE n1.term = n2.term AND n1.term IS NOT NULL AND n1.term <> "" AND id(n1) < id(n2)
                    WITH n1, n2 SKIP {offset} LIMIT {batch_size}
                    MERGE (n1)-[:HAS_SHARED_NAMED_ENTITY]->(n2)
                    MERGE (n2)-[:HAS_SHARED_NAMED_ENTITY]->(n1)
                    RETURN COUNT(*) AS processed
                """
                result = session.run(query)
                processed = result.single()["processed"]
                print(f"Processed {processed} relationships in batch {offset // batch_size + 1}")
                if processed < batch_size:
                    break
                offset += batch_size

if __name__ == "__main__":
    # Initialize Neo4j connection
    conn = Neo4jConnection(uri, user, password)

    # Step 1: Delete old HAS_SHARED_NAMED_ENTITY relationships
    conn.delete_old_named_entity_relationships()

    # Step 2: Get and print duplicated named entities
    duplicated_named_entities = conn.get_duplicated_named_entities()

    # Step 3: Merge duplicate named entities if any duplicates exist
    if duplicated_named_entities:
        conn.merge_duplicate_named_entities()

    # Close the connection
    conn.close()


Connected to Neo4j.
Deleted all old HAS_SHARED_NAMED_ENTITY relationships.
Duplicated named entities and their counts:
Term: 'account', Count: 415
Term: 'exchange', Count: 358
Term: 'clearing', Count: 339
Term: 'interest', Count: 336
Term: 'exposure', Count: 333
Term: 'group', Count: 294
Term: 'position', Count: 288
Term: 'price', Count: 257
Term: 'jurisdiction', Count: 245
Term: 'funds', Count: 239
Term: 'control', Count: 223
Term: 'collateral', Count: 216
Term: 'contract', Count: 214
Term: 'insurer', Count: 211
Term: 'security', Count: 209
Term: 'clearing house', Count: 206
Term: 'issuer', Count: 205
Term: 'liquidity', Count: 203
Term: 'agreement', Count: 182
Term: 'counterparty', Count: 181
Term: 'financial institution', Count: 175
Term: 'fund manager', Count: 163
Term: 'policy', Count: 162
Term: 'fee', Count: 145
Term: 'law', Count: 141
Term: 'principal', Count: 138
Term: 'trade', Count: 132
Term: 'balance', Count: 132
Term: 'future', Count: 131
Term: 'currency', Count: 127
Term: '

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 111, offset: 152} for query: '\n                    MATCH (n1:NamedEntity), (n2:NamedEntity)\n                    WHERE n1.term = n2.term AND n1.term IS NOT NULL AND n1.term <> "" AND id(n1) < id(n2)\n                    WITH n1, n2 SKIP 0 LIMIT 10000\n                    MERGE (n1)-[:HAS_SHARED_NAMED_ENTITY]->(n2)\n                    MERGE (n2)-[:HAS_SHARED_NAMED_ENTITY]->(n1)\n                    RETURN COUNT(*) AS processed\n                '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {descrip

Processed 10000 relationships in batch 1


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 111, offset: 152} for query: '\n                    MATCH (n1:NamedEntity), (n2:NamedEntity)\n                    WHERE n1.term = n2.term AND n1.term IS NOT NULL AND n1.term <> "" AND id(n1) < id(n2)\n                    WITH n1, n2 SKIP 10000 LIMIT 10000\n                    MERGE (n1)-[:HAS_SHARED_NAMED_ENTITY]->(n2)\n                    MERGE (n2)-[:HAS_SHARED_NAMED_ENTITY]->(n1)\n                    RETURN COUNT(*) AS processed\n                '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {des

Processed 10000 relationships in batch 2


...

Processed 10000 relationships in batch 87


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 111, offset: 152} for query: '\n                    MATCH (n1:NamedEntity), (n2:NamedEntity)\n                    WHERE n1.term = n2.term AND n1.term IS NOT NULL AND n1.term <> "" AND id(n1) < id(n2)\n                    WITH n1, n2 SKIP 870000 LIMIT 10000\n                    MERGE (n1)-[:HAS_SHARED_NAMED_ENTITY]->(n2)\n                    MERGE (n2)-[:HAS_SHARED_NAMED_ENTITY]->(n1)\n                    RETURN COUNT(*) AS processed\n                '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {de

Processed 10000 relationships in batch 88


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 111, offset: 152} for query: '\n                    MATCH (n1:NamedEntity), (n2:NamedEntity)\n                    WHERE n1.term = n2.term AND n1.term IS NOT NULL AND n1.term <> "" AND id(n1) < id(n2)\n                    WITH n1, n2 SKIP 880000 LIMIT 10000\n                    MERGE (n1)-[:HAS_SHARED_NAMED_ENTITY]->(n2)\n                    MERGE (n2)-[:HAS_SHARED_NAMED_ENTITY]->(n1)\n                    RETURN COUNT(*) AS processed\n                '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {de

Processed 10000 relationships in batch 89
Processed 1563 relationships in batch 90
Connection to Neo4j closed.


In [25]:
allDone()